In [1]:
import numpy as np
import scqubits as scq

Transmon qubits can be capacitively coupled to a common harmonic mode, realized by an LC oscillator or a transmission-line resonator. The Hamiltonian describing such a composite system is given by:

$$ H=H_{tmon,1}+H_{tmon,2}+\omega_ra^{\dagger} a+\Sigma_{j=1,2} g_jn_j(a+a^{\dagger})$$

where $j=1,2$ enumerates the two transmon qubits,$\omega_r$ is the (angular) frequency of the resonator. Furthermore, $n_j$ is the charge number operator for qubit $j$, and $g_j$ is the coupling strength between qubit and the resonator.



## Creating a HilbertSpace instance

The first step consists of creating the objects describing the individual building blocks of the full Hilbert space. Here, these will be the two transmons and one oscillator:

In [2]:
tmon1 = scq.Transmon(
    EJ=40.0,
    EC=0.2,
    ng=0.3,
    ncut=40,
    truncated_dim=4     # after diagonalization, we will keep 3 levels
)

tmon2 = scq.Transmon(
    EJ=15.0,
    EC=0.15,
    ng=0.0,
    ncut=30,
    truncated_dim=4
)

resonator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4  # up to 3 photons (0,1,2,3)
)

The HilbertSpace object is now created in one of two ways. The first is by utilizing the GUI:

In [3]:
g = 0.1
hilbertspace = scq.HilbertSpace.create()

Output()

The second way of creating a HilbertSpace object is through regular programmatic creation: provide a list of all subsystems and then specify individual interaction terms (see the next subsection for the latter):

In [4]:
hilbertspace = scq.HilbertSpace([tmon1, tmon2, resonator])
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_1]
                    | EJ: 40.0
                    | EC: 0.2
                    | ng: 0.3
                    | ncut: 40
                    | truncated_dim: 4
                    |
                    | dim: 81


Transmon------------| [Transmon_2]
                    | EJ: 15.0
                    | EC: 0.15
                    | ng: 0.0
                    | ncut: 30
                    | truncated_dim: 4
                    |
                    | dim: 61


Oscillator----------| [Oscillator_1]
                    | E_osc: 4.5
                    | l_osc: None
                    | truncated_dim: 4
                    |
                    | dim: 4




## Specifying interactions between systems

Interaction terms describing the coupling between subsystems can be specified in three different ways.

This structure is captured by using the add_interaction method in the following way:


In [5]:
g1 = 0.1  # coupling resonator-tmon1 (without charge matrix elements)
operator1 = tmon1.n_operator()
operator2 = resonator.creation_operator() + resonator.annihilation_operator()

hilbertspace.add_interaction(
    g=g1,
    op1=(operator1, tmon1),
    op2=(operator2, resonator)
)

g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)
hilbertspace.add_interaction(
    g=g2,
    op1=tmon2.n_operator,
    op2=resonator.creation_operator,
    add_hc=True
)

In [6]:
param_name = "Φext/Φ0"
param_vals = np.linspace(0.0, 1.0, 151)

subsys_update_list = [tmon1, tmon2]


def update_hilbertspace(param_val):     # function that shows how Hilbert space components are updated
    tmon1.EJ = 30*np.abs(np.cos(np.pi * param_val))
    tmon2.EJ = 40*np.abs(np.cos(np.pi * param_val * 2))


sweep = scq.ParameterSweep(
    paramvals_by_name={param_name: param_vals},
    evals_count=15,
    hilbertspace=hilbertspace,
    subsys_update_info={param_name: [tmon1, tmon2]},
    update_hilbertspace=update_hilbertspace,
)

Bare spectra:   0%|          | 0/151 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/151 [00:00<?, ?it/s]

Bare spectra:   0%|          | 0/1 [00:00<?, ?it/s]

Dressed spectrum:   0%|          | 0/151 [00:00<?, ?it/s]

In [7]:
explorer = scq.Explorer(sweep=sweep)